In [13]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

ModuleNotFoundError: No module named 'nltk'

In [11]:
df = pd.read_csv('df.csv', sep=';')

In [12]:
df

,text,label
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik
...,...,...
4995,RT @l1DsGGe2xObT3t72dHwqlT58X7jvEYtEnauZIZSYwS...,Politik
4996,Masyarakat yakin bahwa Prabowo-Gibran memiliki...,Politik
4997,imo both are irrational but yg satu jauh lebih...,Ekonomi
4998,@cIIGSdjaPrmAQY1E4gWnLpIZSdyQn8ZMhjJzgOsxfRM= ...,Pertahanan dan Keamanan


In [ ]:
import re

# Precompile regex patterns
PATTERNS = {
    'at': re.compile(r'@\S+'),
    'url': re.compile(r'https?\S+'),
    'bracket': re.compile(r'\[.*?\]'),
    'non_ascii': re.compile(r'[^\x00-\x7F]+'),
    'hashtag': re.compile(r'#\S+'),
    'rt': re.compile(r'\brt\b'),
    'punctuation': re.compile(r'[^\w\s]')
}

# Default masks
DEFAULT_MASKS = {
    'at': 'PERSON_TAG_',
    'url': 'URL_',
    'bracket': 'RT_PERSON_',
    'hashtag': 'HASHTAG_'
}

ADD_STOPWORDS = ['yg']

def clean_text(text, 
               cln_punct=True, 
               mask_person=True, 
               mask_url=True, 
               mask_bracket=True, 
               mask_hashtag=True,
               stopword = True,
               lemma = True,):
    # Lowercase the text
    text = text.lower()
    
    # Mask options
    masks = {
        'at': DEFAULT_MASKS['at'] if mask_person else '',
        'url': DEFAULT_MASKS['url'] if mask_url else '',
        'bracket': DEFAULT_MASKS['bracket'] if mask_bracket else '',
        'hashtag': DEFAULT_MASKS['hashtag'] if mask_hashtag else ''
    }

    # Apply regex substitutions
    text = PATTERNS['at'].sub(masks['at'], text)
    text = PATTERNS['url'].sub(masks['url'], text)
    text = PATTERNS['bracket'].sub(masks['bracket'], text)
    text = PATTERNS['non_ascii'].sub('', text)
    text = PATTERNS['hashtag'].sub(masks['hashtag'], text)
    text = PATTERNS['rt'].sub('', text)
    
    if cln_punct:
        text = PATTERNS['punctuation'].sub('', text)
    if stopword:
        stop_words = stopwords.words('indonesian')
        stop_words = stop_words + ADD_STOPWORDS
        text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    text = ' '.join(text.split())
    return text

# Example usage
df['text_cleaned'] = df['text'].apply(lambda x: clean_text(x,lemma = False,mask_hashtag=False,mask_person=False,mask_url=False,mask_bracket=False))
